# Import Library dan Load Dataset
Pertama-tama kita perlu mengimport library yang dibutuhkan. dataset yang akan digunakan juga kita siapkan

In [412]:
import pandas as pd
import numpy as np

#pengolahan data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#modeling
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

df=pd.read_csv('/content/job.csv')
df.head()

,job_title,company_name,location,start_date,ctc,experience,posted
0,Business Development Associate,Break The Code,Mumbai,Starts Immediately,"₹ 2,00,000 - 3,00,000",0-1 years,3 days ago
1,IELTS/PTE/Spoken English Trainer,DePioneer Education Overseas,Delhi,Starts Immediately,"₹ 3,00,000 - 3,50,000",0-2 years,1 week ago
2,Fashion Designer,Valkyre Clothing,Mumbai,Starts Immediately,"₹ 3,00,000 - 3,50,000",0-2 years,3 days ago\n\n\nBe an early applicant
3,Corporate Sales Executive,Kompass India Information Private Limited,Mumbai,Starts Immediately,"₹ 2,00,000 - 4,00,000",0-5 years,3 weeks ago
4,Junior Python Developer,Extension Technologies Private Limited,Faridabad,Starts Immediately,"₹ 2,40,000 - 3,60,000",0-2 years,2 weeks ago


In [413]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_title     4596 non-null   object
 1   company_name  4596 non-null   object
 2   location      4596 non-null   object
 3   start_date    4596 non-null   object
 4   ctc           4596 non-null   object
 5   experience    4596 non-null   object
 6   posted        4596 non-null   object
dtypes: object(7)
memory usage: 251.5+ KB


#Preprocessing
Tahapan preprocesing ini memuat tentang pengendalian nilai null, pengendalian nilai duplicate, melakukan normalisasi data juga encodeding.


handling missing value

In [414]:
df.isnull().sum()

,0
job_title,0
company_name,0
location,0
start_date,0
ctc,0
experience,0
posted,0


In [415]:
df.dropna(inplace=True)

handling duplicated value

In [416]:
duplicate = df.duplicated().sum()
print('Jumlah nilai duplikat pada data: ', duplicate)

Jumlah nilai duplikat pada data:  94


In [417]:
#Menghapus nilai duplikat
df.drop_duplicates(inplace=True)

# Melihat jumlah duplicate value
duplicate = df.duplicated().sum()
print('Jumlah nilai duplikat pada data: ', duplicate)

Jumlah nilai duplikat pada data:  0


## Pengolahan feature

Melakukan konversi  `Posted` menjadi jumlah hari sejak posting.

In [418]:
def convert_posted(posted_str):
    if 'days' in posted_str:
        return int(posted_str.split()[0])
    elif 'week' in posted_str:
        return int(posted_str.split()[0]) * 7
    elif 'month' in posted_str:
        return int(posted_str.split()[0]) * 30
    else:
        return np.nan #Handling format tidak diketahui


df['posted_days'] = df['posted'].apply(convert_posted)


definisi kolom `ctc` ==> CTC, atau Cost to Company, adalah istilah yang umum digunakan dalam konteks penggajian. Data pada kolom ctc ini berupa object yang ber-rentang, pada tahapan ini kita akan mengambil nilai tengahnya saja.

In [419]:
# Menggunakan lambda untuk menghitung rata-rata CTC
def calculate_avg_ctc(ctc_str):
    try:
        # Membersihkan string dan memisahkan berdasarkan ' - '
        ctc_range = ctc_str.replace('₹', '').replace(',', '').strip().split(' - ')
        # Menghitung rata-rata jika ada dua nilai
        if len(ctc_range) == 2:
            return (int(ctc_range[0]) + int(ctc_range[1])) / 2
        # Jika hanya ada satu nilai, gunakan sebagai rata-rata
        elif len(ctc_range) == 1:
            return int(ctc_range[0])
    except ValueError:
        return np.NaN  # Kembalikan None jika terjadi kesalahan

# Menerapkan fungsi ke kolom ctc
df['ctc'] = df['ctc'].apply(calculate_avg_ctc)



## Encode

Untuk memaksimalkan model maka kolom `experience` diencode tapi sebelumnya kita perlu melakukan pengkategorian kelas nya.

In [420]:
df['experience'] = df['experience'].str.strip()  # Menghapus spasi di awal dan akhir
print(df['experience'].unique())


['0-1 years' '0-2 years' '0-5 years' '0-4 years' '1-3 years' '1-5 years'
 '1 year' '3-5 years' '1-4 years' '2-4 years' '1-2 years' '0-3 years'
 '2-5 years' '0 years' '4-5 years' '2 years' '3 years' '2-3 years'
 '4 years' '3-4 years' '4-6 years' '7-10 years' '10-15 years' '6-10 years'
 '5-7 years' '2-10 years' '3-6 years' '3-7 years']


In [421]:
# Fungsi untuk menghitung nilai tengah dari rentang tahun
def convert_experience(exp_str):
    # Menghapus ' years' dari string
    exp_str = exp_str.replace(' years', '').replace(' year', '')

    if '-' in exp_str:  # Memeriksa jika ada rentang
        years = exp_str.split('-')
        # Menghitung nilai tengah
        return (int(years[0]) + int(years[-1])) / 2
    else:  # Jika hanya satu tahun
        return int(exp_str)

# Mengonversi kolom experience
df['experience_encoded'] = df['experience'].apply(convert_experience)

Label Encoding untuk kolom kategorikal

In [422]:
label_encoders = {}
for col in ['job_title', 'company_name', 'location']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


Scaling data numerik

In [423]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi scaler
scaler = MinMaxScaler()

# Pilih kolom numerik kecuali 'ctc'
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Melakukan scaling pada kolom numerik yang telah dipilih
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])


Cross cek mising value setelah pengolahan fitur jika terdapat missing value kita dapat handling kembali.

In [424]:
df.isnull().sum()

,0
job_title,0
company_name,0
location,0
start_date,0
ctc,176
experience,0
posted,0
posted_days,258
experience_encoded,0


In [425]:
df.dropna(inplace=True)

# Model Deep Learning - Sequential

## Split Dataset

Inisialisasi fitur dan label untuk model kemudian melakukan spliting dataset.

In [426]:
X = df[['job_title', 'company_name', 'location', 'experience_encoded', 'posted_days']]
y = df['ctc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)


## Build Model

Model dibangun menggunakan arsitektur Sequential, di mana setiap layer dihubungkan secara berurutan.

In [427]:
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Layer pertama dengan 64 unit
    Dense(32, activation='relu'),  # Layer kedua dengan 32 unit
    Dense(1, activation='linear') # Output layer untuk regresi (tanpa aktivasi karena regresi)
])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_61 (Dense)                     │ (None, 64)                  │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

## Train Model

Tahapan training dimulai dari mengcompile model yang telah diibuat dengan mengatur optimez yang akan digunakan dalam kasus ini  kita menggunkan sgd dn menilai loss nya menggunakan mse

In [428]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['mse'])

menggunkan Callback: EarlyStopping dari library tensorflow untuk menghentikan pelatihan jika loss tidak membaik setelah 5 epoch juga mempertimbangkan nilai loss nya.


In [429]:
early_stop = EarlyStopping(monitor='loss',
                           patience=5,
                           min_delta=0.001,
                           restore_best_weights=True)

Menggunakan data pelatihan dengan 100 epoch dan batch size 8, serta memantau performanya menggunakan data validasi. Callback early stopping digunakan untuk menghentikan pelatihan lebih awal jika tidak ada peningkatan pada loss, guna mencegah overfitting.

In [430]:
history = model.fit(
    X_train, y_train,
    epochs=100,  # Jumlah epoch, yaitu berapa kali model akan memproses seluruh data pelatihan
    batch_size=8,  # Jumlah sampel yang diproses sebelum model diperbarui
    validation_data=(X_test, y_test),  # mengevaluasi model selama pelatihan
    callbacks=[early_stop]  # menghentikan pelatihan jika tidak ada peningkatan performa
)

Epoch 1/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0174 - mse: 0.0174 - val_loss: 0.0053 - val_mse: 0.0053
Epoch 2/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 3/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0039 - val_mse: 0.0039
Epoch 4/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.0037 - val_mse: 0.0037
Epoch 5/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0025 - mse: 0.0025 - val_loss: 0.0036 - val_mse: 0.0036
Epoch 6/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0035 - val_mse: 0.0035
Epoch 7/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.0034 - val_mse: 0.0034
Epoch 8/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0034 - val_mse: 0.0034
Epoch 9/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/

## Evaluate Model

In [431]:
model.evaluate(X_test, y_test)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0036 - mse: 0.0036


[0.003606781829148531, 0.003606781829148531]

Hasil evaluasi menunjukkan performa model pada data uji sebagai berikut:

Loss (MSE - Mean Squared Error): 0.0037. Ini berarti bahwa rata-rata kuadrat kesalahan antara prediksi model dan nilai sebenarnya adalah 0.0037.

Mean Squared Error (MSE): 0.0037. Metrik ini identik dengan loss dalam kasus ini, karena model diregresikan menggunakan MSE.

Secara umum, semakin kecil nilai MSE, semakin baik model dalam memprediksi nilai target yang mendekati nilai sebenarnya. Nilai 0.0037 menunjukkan bahwa model cukup akurat

**Test prediksi 10 data**

In [432]:
# Ambil prediksi dan nilai sebenarnya
prediksi_flat = prediksi.flatten()
nilai_sebenarnya = y_test[:10].values

# Buat DataFrame
df_hasil = pd.DataFrame({
    'Prediksi': prediksi_flat,
    'Nilai Sebenarnya': nilai_sebenarnya
})

# Tampilkan tabel
print(df_hasil)


   Prediksi  Nilai Sebenarnya
0  0.017774          0.015924
1  0.023475          0.007962
2  0.055643          0.027866
3  0.018149          0.009554
4  0.016431          0.031847
5  0.015438          0.001592
6  0.042435          0.031847
7  0.038942          0.015924
8  0.018742          0.063694
9  0.039064          0.071656
